In [ ]:
import pandas as pd

def rename_cov_metric(metric: str) -> str:
    if metric == 'class cov':
        return 'class'
    if metric == 'method cov':
        return 'method'
    if metric == 'branch cov':
        return 'branch'
    if metric == 'instruction cov':
        return 'instruction'

data1 = pd.read_csv('seed-corpus_fuzzing_manual-inspection.csv',
                    usecols=['program', 'class cov', 'method cov', 'branch cov', 'instruction cov'])
data1['input_corpus'] = 'seed corpus + fuzzing + manual inspection'

data = pd.concat([data1])
data = pd.melt(data.reset_index(),
               id_vars=['program', 'input_corpus'],
               value_vars=['class cov', 'method cov', 'branch cov', 'instruction cov'],
               var_name='metric',
               value_name='metric_value')
data['metric'] = data['metric'].map(rename_cov_metric)
data['metric_value'] = data['metric_value'] * 100

data.set_index(['program', 'input_corpus', 'metric'])
data

In [ ]:
import seaborn as sns
from matplotlib import pyplot, ticker
import math

sns.set_theme(style="whitegrid", palette='colorblind')
sns.color_palette("colorblind")

# df = data.reindex(pd.MultiIndex.from_product(data.index.levels)).reset_index()
df = data.reset_index()

grid = sns.FacetGrid(data=df,
                     row = 'metric'
                     )
grid.map_dataframe(sns.barplot,
                   y='program',
                   x='metric_value',
                   hue='input_corpus',
                   palette='colorblind')
for axis in grid.axes:
    axis[0].xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:d}%'.format(int(x))))

grid.set_axis_labels("Metric", "Program")
grid.set_titles(col_template="{col_name}", row_template="{row_name}")


pyplot.savefig('coverage.png', dpi=300)
pyplot.show()